In [1]:
! pip install tensorboard

In [2]:
import os

# Check if the notebook is running on Colab
if 'COLAB_GPU' in os.environ:
    # This block will run only in Google Colab
    IN_COLAB = True
    print("Running on Google Colab. Cloning the repository.")
    !git clone https://github.com/pedro15sousa/R244-wm-load-balancing.git
    %cd R244-wm-load-balancing/notebooks
else: 
    # This block will run if not in Google Colab
    IN_COLAB = False
    print("Not running on Google Colab. Assuming local environment.")

Not running on Google Colab. Assuming local environment.


In [3]:
import sys
sys.path.append('..')  # This adds the parent directory (main_folder) to the Python path

In [5]:
import argparse
from os.path import join, exists
from os import mkdir

import torch
import torch.utils.data
from torch import optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter

from models.vae import VAE

from utils.misc import save_checkpoint
from utils.misc import LSIZE, RED_SIZE, ASIZE
## WARNING : THIS SHOULD BE REPLACE WITH PYTORCH 0.5
from utils.learning import EarlyStopping
from utils.learning import ReduceLROnPlateau
from data.loaders import RolloutObservationDataset

## Initial Setup

In [6]:
# parser = argparse.ArgumentParser(description='VAE Trainer')
# parser.add_argument('--batch-size', type=int, default=32, metavar='N',
#                     help='input batch size for training (default: 32)')
# parser.add_argument('--epochs', type=int, default=1000, metavar='N',
#                     help='number of epochs to train (default: 1000)')
# parser.add_argument('--logdir', type=str, help='Directory where results are logged')
# parser.add_argument('--noreload', action='store_true',
#                     help='Best model is not reloaded if specified')
# parser.add_argument('--nosamples', action='store_true',
#                     help='Does not save samples during training if specified')

# args = parser.parse_args()

args = {
    'batch_size': 32,   # input batch size for training (default: 32)
    'epochs': 1,     # number of epochs to train (default: 1000)
    'logdir': '../exp_dir',  # Directory where results are logged
    'noreload': False,  # Set True if best model is not to be reloaded
    'nosamples': False,  # Set True if samples are not to be saved during training
}

In [7]:
cuda = torch.cuda.is_available()

torch.manual_seed(123)
# Fix numeric divergence due to bug in Cudnn
torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if cuda else "cpu")

# 51 is the number of dimensions in my space: 50 servers + 1 job size
model = VAE(ASIZE+1, LSIZE).to(device)
optimizer = optim.Adam(model.parameters())
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5)
earlystopping = EarlyStopping('min', patience=30)

In [8]:
def load_data():
    dataset_train = RolloutObservationDataset('../datasets/loadbalancing', train=True)
    dataset_test = RolloutObservationDataset('../datasets/loadbalancing',train=False)

    train_loader = torch.utils.data.DataLoader(
        dataset_train, batch_size=args['batch_size'], shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(
        dataset_test, batch_size=args['batch_size'], shuffle=True, num_workers=2)

    return dataset_test, dataset_train, test_loader, train_loader

## Loss Function

In [9]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logsigma):
    """ VAE loss function """
    BCE = F.mse_loss(recon_x, x, size_average=False)
    # print(mu)
    # print(logsigma)
    # print(recon_x)
    # print(x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + 2 * logsigma - mu.pow(2) - (2 * logsigma).exp())
    return BCE + KLD

## Training Method

In [10]:
def train(epoch):
    """ One training epoch """
    model.train()
    dataset_train.load_next_buffer()
    train_loss = 0
    print("\nTrain Loader size: ", len(train_loader))
    for batch_idx, data in enumerate(train_loader):
        data = data.float().to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    train_loss /= len(train_loader.dataset)
    return train_loss

## Testing Method

In [11]:
def test():
    """ One test epoch """
    model.eval()
    dataset_test.load_next_buffer()
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.float().to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss

In [12]:
def test_vae_single_sample(model, test_loader, device):
    model.eval()
    with torch.no_grad():
        # Get a single batch from the test loader
        data = next(iter(test_loader))[0].to(device)
        
        # Select the first sample from the batch
        sample = data.unsqueeze(0)  # Add batch dimension

        print(sample)
        print("Sample shape:", sample.shape)  # Add this line to check the shape

        # Forward pass through the VAE
        recon, mu, logsigma = model(sample)

        sigma = logsigma.exp()
        eps = torch.randn_like(sigma)
        z = eps.mul(sigma).add_(mu)

        # Convert tensors to numpy arrays for visualization
        sample_np = sample.squeeze().cpu().numpy()
        recon_np = recon.squeeze().cpu().numpy()
        z_np = z.squeeze().cpu().numpy()
        
        print("\nOriginal Sample: ", sample_np)
        print("Reconstructed Sample: ", recon_np)
        print("Latent Space Sample: ", z_np)

## Main Loop

In [13]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir ../exp_dir/MNIST/vae/tensorboard

In [15]:
if __name__ == "__main__":
    # Initialize the TensorBoard writer
    logdir = args['logdir']
    writer = SummaryWriter(os.path.join(logdir, 'vae/tensorboard'))

    # load data
    dataset_test, dataset_train, test_loader, train_loader = load_data()

    # check vae dir exists, if not, create it
    vae_dir = join(args['logdir'], 'vae')
    if not exists(vae_dir):
        mkdir(vae_dir)
        mkdir(join(vae_dir, 'samples'))

    reload_file = join(vae_dir, 'best.tar')
    if not args['noreload'] and exists(reload_file):
        state = torch.load(reload_file)
        print("Reloading model at epoch {}"
            ", with test error {}".format(
                state['epoch'],
                state['precision']))
        model.load_state_dict(state['state_dict'])
        optimizer.load_state_dict(state['optimizer'])
        scheduler.load_state_dict(state['scheduler'])
        earlystopping.load_state_dict(state['earlystopping'])

        # Assuming you have a trained VAE model, a test_loader, and a device (e.g., 'cuda' or 'cpu')
        test_vae_single_sample(model, test_loader, device)


    cur_best = None

    for epoch in range(1, args['epochs'] + 1):
        train_loss = train(epoch)
        test_loss = test()

        # TensorBoard logging
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/test', test_loss, epoch)

        scheduler.step(test_loss)
        earlystopping.step(test_loss)

        # checkpointing
        best_filename = join(vae_dir, 'best.tar')
        filename = join(vae_dir, 'checkpoint.tar')
        is_best = not cur_best or test_loss < cur_best
        if is_best:
            cur_best = test_loss

        save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'precision': test_loss,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'earlystopping': earlystopping.state_dict()
        }, is_best, filename, best_filename)

        # if not args['nosamples']:
        #     with torch.no_grad():
        #         sample = torch.randn(RED_SIZE, LSIZE).to(device)
        #         sample = model.decoder(sample).cpu()
        #         save_image(sample.view(64, 3, RED_SIZE, RED_SIZE),
        #                 join(vae_dir, 'samples/sample_' + str(epoch) + '.png'))

        if earlystopping.stop:
            print("End of Training because of early stopping at epoch {}".format(epoch))
            break

Loading file buffer ...: 100%|██████████| 1000/1000 
Loading file buffer ...: 100%|██████████| 1000/1000 


Reloading model at epoch 1, with test error 0.7008767317237854
tensor([[1.0000, 0.3363, 0.2604, 0.0525, 0.3730, 0.0000, 0.0787, 0.1480, 0.0000,
         0.0000, 0.0513]])
Sample shape: torch.Size([1, 11])

Original Sample:  [1.         0.3362939  0.26039565 0.05248284 0.37303188 0.
 0.07872427 0.14796124 0.         0.         0.0512717 ]
Reconstructed Sample:  [0.70486295 0.52887386 0.35667124 0.26748252 0.17934649 0.12874219
 0.10001577 0.07928371 0.07021698 0.06133177 0.03459665]
Latent Space Sample:  [ 0.35530996 -0.03078336  0.60374904  1.103252   -0.701139   -0.02517617
  0.2703277   1.7393682   0.07784095 -0.27177042 -1.8276705 ]


Loading file buffer ...: 100%|██████████| 1000/1000 


Train Loader size:  31250


Train Epoch: 1 [0/1000000 (0%)]	Loss: 0.641733
Train Epoch: 1 [640/1000000 (0%)]	Loss: 0.660052
Train Epoch: 1 [1280/1000000 (0%)]	Loss: 0.693514
Train Epoch: 1 [1920/1000000 (0%)]	Loss: 0.659740
Train Epoch: 1 [2560/1000000 (0%)]	Loss: 0.613493
Train Epoch: 1 [3200/1000000 (0%)]	Loss: 0.628386
Train Epoch: 1 [3840/1000000 (0%)]	Loss: 0.621188
Train Epoch: 1 [4480/1000000 (0%)]	Loss: 0.672482
Train Epoch: 1 [5120/1000000 (1%)]	Loss: 0.684296
Train Epoch: 1 [5760/1000000 (1%)]	Loss: 0.639472
Train Epoch: 1 [6400/1000000 (1%)]	Loss: 0.716257
Train Epoch: 1 [7040/1000000 (1%)]	Loss: 0.620353
Train Epoch: 1 [7680/1000000 (1%)]	Loss: 0.724332
Train Epoch: 1 [8320/1000000 (1%)]	Loss: 0.706579
Train Epoch: 1 [8960/1000000 (1%)]	Loss: 0.678379
Train Epoch: 1 [9600/1000000 (1%)]	Loss: 0.714814
Train Epoch: 1 [10240/1000000 (1%)]	Loss: 0.619089
Train Epoch: 1 [10880/1000000 (1%)]	Loss: 0.740326
Train Epoch: 1 [11520/1000000 (1%)]	Loss: 0.809430
Train Epoch: 1 [12160/1000000 (1%)]	Loss: 0.604790


KeyboardInterrupt: 